__собираем предложения о продаже домов__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/house/avito' 
# cian_raw_data_path = 'data/raw/cian'
result_file_path = 'data/data_house.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/house/avito/avito_2022-09-15_11-54_raw_sevastopol.xlsx',
 'data/raw/house/avito/avito_2022-09-15_14-42_raw_bahchisaray.xlsx',
 'data/raw/house/avito/avito_2022-09-15_14-47_raw_sudak.xlsx',
 'data/raw/house/avito/avito_2022-09-16_15-21_raw_sevastopol.xlsx',
 'data/raw/house/avito/avito_2022-09-16_15-22_raw_bahchisaray.xlsx',
 'data/raw/house/avito/avito_2022-09-16_15-24_raw_sudak.xlsx',
 'data/raw/house/avito/avito_2022-09-19_12-40_raw_sudak.xlsx']

In [7]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

5709


In [8]:
# data_avito

In [9]:
from lib.avito import AvitoDataCleanerRealtyHouse

data_avito = AvitoDataCleanerRealtyHouse().transform( data_avito )

print(len(data_avito))

5709


In [10]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,house_area,is_part,is_townhouse,is_SNT,land_area,priceM,land_size_category,house_size_category
2884,2167418549,"дом 72 м² на участке 4,3 сот.",5990000,,"садоводческое товарищество Электрон-2, 74",Я собственник. Продается участок с шикарным расположением и двухэтажным домом 72 метра в самом сердце мыса Фиолент.\nУчасток прямоугольной формы обнесён забором с автоматическими воротами.\nЭлектричество разведено по дому. Городская вода на участке. Газ в бл,2,2022-09-16 15:21:10.419,sevastopol,72.00,False,False,True,4.30,5.99,4-8,70-150
3531,2182696134,дом 170 м² на участке 4 сот.,10000000,,"СТ Конус, Фиолентовское ш., 64к1","Продам дом с возможностью ипотеки, материнского капитала, сертификата и другие способы реализации. Дом располагается рядом с остановкой (первая линия) и 5 мин. Пешком до моря. Удобный заезд с Фиолентовского шоссе, где в 10 ми. До ТЦ Муссон, Метро и 20 мин.",15,2022-09-16 15:21:10.419,sevastopol,170.00,False,False,True,4.00,10.00,2-4,150-300
3822,2569906544,"дом 45 м² на участке 4,5 сот.",3600000,,"садоводческое товарищество Дергачи-4, 157","Продаётся Дом 45м. Кв., СТ«Дергачи-4» на участке 4,5 соток, две комнаты, кухня, прекрасный сад. Коммуникации: электричество, городская вода, газ по забору (подключение отдельно). В шаговой доступности: Остановка общественного транспорта, тц, магазины. Доку",21,2022-09-16 15:21:10.419,sevastopol,45.00,False,False,True,4.50,3.60,4-8,30-50


In [11]:
data_avito['place'].unique()

array(['sevastopol', 'bahchisaray', 'sudak'], dtype=object)

In [12]:
assert len(data_avito)>0

In [13]:
data_avito.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5709 entries, 0 to 5708
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   avito_id             5709 non-null   int64         
 1   title                5709 non-null   object        
 2   price                5709 non-null   int64         
 3   obj_name             5709 non-null   object        
 4   adr                  5709 non-null   object        
 5   description          5709 non-null   object        
 6   avito_page           5709 non-null   int64         
 7   ts                   5709 non-null   datetime64[ns]
 8   place                5709 non-null   object        
 9   house_area           5709 non-null   float64       
 10  is_part              5709 non-null   bool          
 11  is_townhouse         5709 non-null   bool          
 12  is_SNT               5709 non-null   bool          
 13  land_area            5709 non-nul

In [14]:
data_avito.isnull().any()

avito_id               False
title                  False
price                  False
obj_name               False
adr                    False
description            False
avito_page             False
ts                     False
place                  False
house_area             False
is_part                False
is_townhouse           False
is_SNT                 False
land_area              False
priceM                 False
land_size_category      True
house_size_category     True
dtype: bool

In [15]:
data_avito[ data_avito['land_size_category'].isnull() ]

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,house_area,is_part,is_townhouse,is_SNT,land_area,priceM,land_size_category,house_size_category
2283,2337799298,,11900000,,Восточная ул.,"Предлагаю 1/2 дома общей площадью 120 кв. М. В центре города в отличном районе для проживания на земельном участке 1.6 сотки. Вся инфраструктура рядом, транспортные развязки в шаговой доступности. При этом дом расположен в тихом месте, с каждого этажа дома",47,2022-09-15 11:53:29.976,sevastopol,0.00,True,False,False,0.00,11.90,NaN,NaN
2614,2034310938,,5000000,,"с. Танковое, ул. Ходырева","Собственник. Срочно продам дом 74,3 кв м на земельном участке 814 м кв. Санузел и туалет в доме, 2 спальни. 1965 года постройки, камень бут, очень тёплый зимой и прохладный летом. Газ, приборы учета на все коммуникации. В дом есть 2 входа. Также на участке",4,2022-09-15 14:42:27.254,bahchisaray,0.00,False,False,False,0.00,5.00,NaN,NaN
2624,2391096190,,3000000,,"с. Растущее, Садовая ул., 8","Продам Дом 75 КВ. М, 10 Соток Земли Ижс Продам дом в живописном районе Крыма, в селе Растущее 10 соток земли, 75 кв. М, 4 комнаты, коммуникации: свет, газ, вода, скважина, отопление Агв. Дом чистый, уютный, заходи живи. Участок ровный, сад, весь обработанн",4,2022-09-15 14:42:27.254,bahchisaray,0.00,False,False,True,0.00,3.00,NaN,NaN
2733,2418227584,,5500000,,"Платановый квартал, 21",Недострой (Ижс).\nЗемля 5.2 сотки (собственность).\nВсе вопросы по телефону.,1,2022-09-15 14:47:45.056,sudak,0.00,False,False,False,0.00,5.50,NaN,NaN
5076,2337799298,,11900000,,Восточная ул.,"Предлагаю 1/2 дома общей площадью 120 кв. М. В центре города в отличном районе для проживания на земельном участке 1.6 сотки. Вся инфраструктура рядом, транспортные развязки в шаговой доступности. При этом дом расположен в тихом месте, с каждого этажа дома",48,2022-09-16 15:21:10.419,sevastopol,0.00,True,False,False,0.00,11.90,NaN,NaN
5213,2391096190,,3000000,,"с. Растущее, Садовая ул., 8","Продам Дом 75 КВ. М, 10 Соток Земли Ижс Продам дом в живописном районе Крыма, в селе Растущее 10 соток земли, 75 кв. М, 4 комнаты, коммуникации: свет, газ, вода, скважина, отопление Агв. Дом чистый, уютный, заходи живи. Участок ровный, сад, весь обработанн",1,2022-09-16 15:22:23.263,bahchisaray,0.00,False,False,True,0.00,3.00,NaN,NaN
5384,2034310938,,5000000,,"с. Танковое, ул. Ходырева","Собственник. Срочно продам дом 74,3 кв м на земельном участке 814 м кв. Санузел и туалет в доме, 2 спальни. 1965 года постройки, камень бут, очень тёплый зимой и прохладный летом. Газ, приборы учета на все коммуникации. В дом есть 2 входа. Также на участке",4,2022-09-16 15:22:23.263,bahchisaray,0.00,False,False,False,0.00,5.00,NaN,NaN
5511,2418227584,,5500000,,"Платановый квартал, 21",Недострой (Ижс).\nЗемля 5.2 сотки (собственность).\nВсе вопросы по телефону.,1,2022-09-16 15:24:24.472,sudak,0.00,False,False,False,0.00,5.50,NaN,NaN
5645,2418227584,,5500000,,"Платановый квартал, 21",Недострой (Ижс).\nЗемля 5.2 сотки (собственность).\nВсе вопросы по телефону.,2,2022-09-19 12:40:34.305,sudak,0.00,False,False,False,0.00,5.50,NaN,NaN


---

In [16]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

5709


## обновляем таблицу адресов

In [17]:
# data_ = data.sample(15).reset_index(drop=True)
# data_[['adr']]

places = { 
# поле 'place' и название города
'bahchisaray': 'Бахчисарай,',
      'sudak': 'Судак,',
  'sevastopol': 'Севастополь,',
}

def norm_place(d,places=places):
    if not d['place'] in places: return d['adr_']
    p = places[ d['place'] ]
    return d['adr_'] if d['adr_'].find(p)>-1 else p+d['adr_']

In [18]:
from lib.locator import AddressTransformer

atr = AddressTransformer()
data['adr_'] = data['adr'].progress_apply(atr.transform)
data['adr_'] = data[['adr_','place']].progress_apply(norm_place,axis=1)
data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

data[['adr_orig','adr']].sample(10)

  0%|          | 0/5709 [00:00<?, ?it/s]

  0%|          | 0/5709 [00:00<?, ?it/s]

,adr_orig,adr
4360,"садоводческое товарищество Орбита, 12","Севастополь,садоводческое товарищество Орбита, 12"
3127,"садовое товарищество Мидэус, 140","Севастополь,садовое товарищество Мидэус, 140"
164,пос. городского типа Кача,"Севастополь,пос. городского типа Кача"
3390,садоводческое некоммерческое товарищество Тополек,"Севастополь,садоводческое некоммерческое товарищество Тополек"
802,"Севастополь, СТ Море","Севастополь, СТ Море"
3703,"ул. Челнокова, 15/34","Севастополь,ул. Челнокова, 15/34"
5658,СНТ Солнечная Долина,"Судак,СНТ Солнечная Долина"
3526,"Пластунская ул., 50","Севастополь,Пластунская ул., 50"
5602,"с. Дачное, Миндальная ул., 57","Судак,с. Дачное, Миндальная ул., 57"
4120,садоводческое товарищество Мираж,"Севастополь,садоводческое товарищество Мираж"


In [19]:
# from lib.locator import GeocoderSimpleOSM
# locator = GeocoderSimpleOSM()

from lib.locator import GeocoderYandex
from yandex_keys import keys
locator = GeocoderYandex(keys[2])

atr = AddressTransformer()

loc_file_path = 'data/location.pkl' # таблица адресов
loc = pd.read_pickle(loc_file_path)
# loc = pd.DataFrame([],columns=['adr','latitude','longitude',])
logging.info(f'LocationUpdater: {len(loc)} addresses in index')

# собираем все адреса в один список
loc = loc.merge(
        data[['adr']].drop_duplicates(),
        how='outer',
        on='adr',
    ).drop_duplicates().reset_index(drop=True)

logging.info(f'LocationUpdater: {len(loc)} addresses total')

# собираем все адреса с геопозицией
loc_def = loc[~loc['latitude'].isna()].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_def)} addresses defined')

# собираем все адреса без геопозиции
loc_undef = loc[loc['latitude'].isna()][['adr']].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_undef)} addresses undefined')

# определяем координаты по строке адреса
loc_undef = loc_undef.join( loc_undef['adr'].progress_apply(locator.transform).apply(pd.Series) ).dropna()     
logging.info(f'LocationUpdater: {len(loc_undef)} new addresses found')

loc = pd.concat([loc_def,loc_undef]).reset_index(drop=True) 

print( len(loc) )
loc.sample(3)

[INFO    ] 2022-09-20 01:32:15 | LocationUpdater: 4566 addresses in index
[INFO    ] 2022-09-20 01:32:15 | LocationUpdater: 5991 addresses total
[INFO    ] 2022-09-20 01:32:15 | LocationUpdater: 4566 addresses defined
[INFO    ] 2022-09-20 01:32:15 | LocationUpdater: 1425 addresses undefined


  0%|          | 0/1425 [00:00<?, ?it/s]

[INFO    ] 2022-09-20 01:38:56 | LocationUpdater: 1425 new addresses found
5991


,latitude,longitude,adr
2069,44.588173,33.45557,"Севастополь, улица Астана Кесаева, 14"
1307,44.60,33.52,"Севастополь, район Ленинский, улица Очаковцев, 35"
68,44.60,33.50,"Севастополь,Краснодарская улица , 11"


In [20]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

## дополняем данные геометкой

In [21]:
# data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

data = data.merge(loc,on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isna() ] ) )

assert len(data)>0
data.to_pickle(result_file_path)

data.info()

всего записей: 5709
записей без геометки: 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5709 entries, 0 to 5708
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   avito_id             5709 non-null   int64         
 1   title                5709 non-null   object        
 2   price                5709 non-null   int64         
 3   obj_name             5709 non-null   object        
 4   adr_orig             5709 non-null   object        
 5   description          5709 non-null   object        
 6   avito_page           5709 non-null   int64         
 7   ts                   5709 non-null   datetime64[ns]
 8   place                5709 non-null   object        
 9   house_area           5709 non-null   float64       
 10  is_part              5709 non-null   bool          
 11  is_townhouse         5709 non-null   bool          
 12  is_SNT               5709 non-null   bool     